In [44]:
import joblib
import torch
import torch.nn as nn
import pandas as pd

In [25]:
class MovieRecommender(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(MovieRecommender, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


In [68]:
movies = pd.read_csv("Data/movies.csv")
ratings = pd.read_csv("Data/ratings.csv")
links = pd.read_csv("Data/links.csv")
svd_model = joblib.load('Data/svd_model.pkl')
content_based_model = joblib.load(r'Data/content_based_model.pkl')
user_encoder = joblib.load('Data/user_encoder.pkl')
movie_encoder = joblib.load('Data/movie_encoder.pkl')
scaler = joblib.load('Data/scaler.pkl')

In [70]:
def get_hybrid_recommendations(user_id, top_n=10, alpha=0.5):
    user_idx = user_encoder.transform([user_id])[0]
    unrated_movies =  movies[~movies['movieId'].isin(ratings['movieId'][ratings['userId']==user_id])]
    candidate_movies = unrated_movies.copy()
    candidate_movies['userId'] = user_idx

    feature_columns = ['userId'] + [col for col in movies.columns if col not in ['movieId', 'title']]
    X_candidate = torch.tensor(scaler.transform(candidate_movies[feature_columns]), dtype=torch.float32)

    content_based_model.eval()
    with torch.no_grad():
        candidate_movies['content_score'] = content_based_model(X_candidate).numpy().flatten()
    
    candidate_movies['svd_score'] = candidate_movies['movieId'].apply(lambda x: svd_model.predict(user_idx, x).est)
    candidate_movies['hybrid_score'] = alpha * candidate_movies['content_score'] + (1 - alpha) * candidate_movies['svd_score']
    
    top_recommendations = candidate_movies.sort_values(by='hybrid_score', ascending=False).head(top_n)
    
    return top_recommendations[['movieId', 'title', 'content_score', 'svd_score', 'hybrid_score']]

# Example Usage
user_id = 1 # Replace with actual user ID
recommendations = get_hybrid_recommendations(user_id)
print("Top Hybrid Recommendations:")
recommendations

Top Hybrid Recommendations:


E:\Users\mazen\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


,movieId,title,content_score,svd_score,hybrid_score
898,898,Star Wars: Episode V - The Empire Strikes Back...,4.250794,5.000000,4.625397
527,527,"Aristocats, The (1970)",4.116347,5.000000,4.558173
6016,6016,Kiss Kiss Bang Bang (2005),4.096378,5.000000,4.548189
5618,5618,Dark Portals: The Chronicles of Vidocq (Vidoc...,4.073554,4.986054,4.529804
2571,2571,Teenage Mutant Ninja Turtles II: The Secret of...,4.092148,4.908234,4.500191
3275,3275,"10th Victim, The (La decima vittima) (1965)",4.086528,4.877520,4.482024
2028,2028,"South Park: Bigger, Longer and Uncut (1999)",4.061604,4.872762,4.467183
913,913,"Third Man, The (1949)",4.087634,4.798630,4.443132
4973,4973,Oklahoma! (1955),3.879189,5.000000,4.439594
951,951,Chinatown (1974),4.066905,4.782972,4.424938


In [72]:
import requests
# Implement the function to take new_id value map it back to original id and then query the API and return the movie title
def get_titles(movie_id):
    #print("index_id",index_id)
    imdbId = links['imdbId'][links['movieId']==movie_id].values[0]
    #print("imdbId",imdbId)

    #url = "https://www.imdb.com/title/tt0114709/"
    # Define headers to make the request look like it's coming from a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }
    
    response = requests.get(f"https://www.omdbapi.com/?i=tt0{imdbId}&apikey=56280332", headers=headers)
    
    if response.status_code == 200:
        #print("Request successful!")
        data = response.json()
        #print(data)  # This will print the HTML content of the page
        return data.get("Title", "Title not found")
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return f"Failed to retrieve the page. Status code: {response.status_code}"

In [74]:
get_titles(50)

'The Usual Suspects'